# 6. Gibt es einen Zusammenhang dazwischen, wie man zuletzt gegen einen bestimmten Gegner gespielt hat, und wie das nächste Spiel gegen diesen Gegner ausgehen wird?

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
matches = pd.read_csv('../../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../../data/raw/atp_players_till_2022.csv')
rankings = pd.read_csv('../../data/raw/atp_rankings_till_2022.csv')

In [3]:
# Identifizierung von Spielerpaaren und deren historischen Begegnungen
# Wir erstellen ein DataFrame, das Paare von Spielern und das Ergebnis ihrer Spiele beinhaltet.

# Erstellen einer eindeutigen Identifikation für jedes Spielerpaar
matches['player_pair'] = matches.apply(lambda x: tuple(sorted([x['winner_id'], x['loser_id']])), axis=1)

# Auswählen relevanter Spalten
player_pairs_data = matches[['tourney_date', 'player_pair', 'winner_id']]

# Sortieren der Daten nach Spielerpaaren und Datum
player_pairs_data = player_pairs_data.sort_values(by=['player_pair', 'tourney_date'])

# Anzeigen der ersten paar Zeilen der neuen Tabelle
player_pairs_data


,tourney_date,player_pair,winner_id
1694,19680304,"(100001, 100039)",100001
1676,19680304,"(100001, 100073)",100073
9254,19700325,"(100001, 100074)",100074
6193,19690317,"(100001, 100119)",100119
37449,19770207,"(100001, 100324)",100324
...,...,...,...
21325,19730604,"(211289, 211298)",211298
21306,19730604,"(211296, 211300)",211296
21435,19731119,"(211306, 211311)",211306
17168,19720117,"(211306, 211376)",211376


In [4]:
# Analyse des Einflusses des letzten Spiels auf das nächste Spiel

# Berechnung des Gewinners des vorherigen Spiels für jedes Spielerpaar
player_pairs_data['prev_winner_id'] = player_pairs_data.groupby('player_pair')['winner_id'].shift(1)

# Entfernen der ersten Begegnungen, da sie kein vorheriges Spiel haben
player_pairs_data.dropna(subset=['prev_winner_id'], inplace=True)

# Umwandlung der Gewinner-ID in einen booleschen Wert: True, wenn der Gewinner des vorherigen Spiels auch das aktuelle Spiel gewonnen hat
player_pairs_data['prev_winner_won_again'] = player_pairs_data['winner_id'] == player_pairs_data['prev_winner_id']

# Berechnung der Häufigkeit, mit der der Gewinner des vorherigen Spiels auch das nächste Spiel gewinnt
prev_winner_success_rate = player_pairs_data['prev_winner_won_again'].mean()

prev_winner_success_rate

0.605718705495261

In [5]:
player_pairs_data

,tourney_date,player_pair,winner_id,prev_winner_id,prev_winner_won_again
6557,19690808,"(100002, 100005)",100002,100005.0,False
2729,19681004,"(100002, 100029)",100029,100029.0,True
6077,19690804,"(100002, 100029)",100029,100029.0,True
6552,19690808,"(100002, 100029)",100029,100029.0,True
6554,19690808,"(100002, 100030)",100002,100030.0,False
...,...,...,...,...,...
186901,20220627,"(206173, 207989)",206173,207989.0,False
187081,20220725,"(206173, 207989)",206173,206173.0,True
187461,20220829,"(206173, 207989)",207989,206173.0,False
187929,20221107,"(206909, 208103)",206909,206909.0,True


In [6]:
# Berechnung der Anzahl der Fälle, in denen der vorherige Gewinner bzw. Verlierer das nächste Spiel gewinnt
prev_winner_wins = player_pairs_data['prev_winner_won_again'].sum()
prev_loser_wins = len(player_pairs_data) - prev_winner_wins

# Erstellung eines DataFrame für die Daten
data = pd.DataFrame({
    'Ergebnis des nächsten Spiels': ['Vorheriger Gewinner gewinnt', 'Vorheriger Verlierer gewinnt'],
    'Anzahl der Spiele': [prev_winner_wins, prev_loser_wins]
})

# Normierung der Daten
total_games = data['Anzahl der Spiele'].sum()
data['Anzahl der Spiele'] = data['Anzahl der Spiele'] / total_games

# Erstellung des Balkendiagramms
fig = px.bar(data, x='Ergebnis des nächsten Spiels', y='Anzahl der Spiele', 
             title='Vergleich der Ergebnisse auf Basis des vorherigen Spiels (normiert)')

# Anzeigen des Diagramms
fig.show()

Das Balkendiagramm zeigt den Vergleich zwischen der Anzahl der Spiele, in denen der Gewinner des vorherigen Spiels auch das nächste Spiel gewinnt (links), und der Anzahl der Spiele, in denen der Verlierer des vorherigen Spiels das nächste Spiel gewinnt (rechts). Wie zu sehen ist, gibt es eine höhere Tendenz, dass der Gewinner des letzten Spiels auch das nächste Spiel gegen denselben Gegner gewinnt, aber es gibt immer noch eine signifikante Anzahl von Fällen, in denen der Verlierer des vorherigen Spiels das nächste Spiel gewinnt.

## Jetzt schauen wir uns an was passiert, wenn die Spielerpaarungen nach Anzahl Spiele gegeneinander unterschieden werden

In [7]:
matches = pd.read_csv('../../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../../data/raw/atp_players_till_2022.csv')
rankings = pd.read_csv('../../data/raw/atp_rankings_till_2022.csv')
mpdf = matches.merge(players, left_on='winner_id', right_on='player_id')

In [8]:
mpdf = mpdf.drop(columns=['surface', 'draw_size', 'tourney_level','winner_seed', 'winner_entry','winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
                          'loser_seed', 'loser_entry','loser_hand','loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round','minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon','w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt','l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced','winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points', 'player_id', 'name_first', 'name_last', 'hand', 'dob', 'ioc', 'height', 'wikidata_id'])
mpdf['tourney_date'] = pd.to_datetime(mpdf['tourney_date'], format='%Y%m%d')

In [9]:
mpdf.head()

,tourney_id,tourney_name,tourney_date,match_num,winner_id,winner_name,loser_id,loser_name
0,1968-2029,Dublin,1968-07-08,270,112411,Douglas Smith,110196,Peter Ledbetter
1,1968-7308,Adelaide,1968-11-23,249,112411,Douglas Smith,202883,Adi Kourim
2,1968-9341,Brisbane,1968-12-09,257,112411,Douglas Smith,109814,Donald Dell
3,1968-2029,Dublin,1968-07-08,271,126914,Louis Pretorius,209536,Maurice Pollock
4,1968-2029,Dublin,1968-07-08,272,209523,Cecil Pedlow,209535,John Mulvey


### Funktionen

In [10]:
def count_player_pairings(df, min_games, max_games):
    # Erstellen einer Erkennung für die Paarungen
    df['pairing'] = df.apply(lambda x: tuple(sorted([x['winner_name'], x['loser_name']])), axis=1)

    
    pairings_count = df.groupby('pairing').size().reset_index(name='games_count')

    # Filter um nur die Paarungen zu erhalten, die zwischen min_games und max_games gespielt wurden
    frequent_pairings = pairings_count[
        (pairings_count['games_count'] >= min_games) &
        (pairings_count['games_count'] <= max_games)
    ].copy()  # Create a copy to avoid SettingWithCopyWarning

    
    new_columns = pd.DataFrame(frequent_pairings['pairing'].tolist(), index=frequent_pairings.index)
    frequent_pairings['Player1'] = new_columns[0]
    frequent_pairings['Player2'] = new_columns[1]

    # Drop the 'pairing' column as it is no longer needed
    frequent_pairings = frequent_pairings.drop('pairing', axis=1)

    return frequent_pairings


In [11]:
# Erstellen einer Funktion, die die Ergebnisse der Paarungen analysiert 
def analyze_outcomes_neutral(df, pairings):
    
    neutral_results = []

    # Analyse für jede Paarung
    for index, row in pairings.iterrows():
        player1, player2 = row['Player1'], row['Player2']

        
        matches = df[(df['winner_name'].isin([player1, player2])) & (df['loser_name'].isin([player1, player2]))]

        # Sortiere nach Datum, um die Reihenfolge der Spiele zu erhalten
        matches = matches.sort_values(by='tourney_date')

        
        counters = {
            player1: {'last_result': None, 'win_after_win': 0, 'lose_after_win': 0, 'win_after_lose': 0, 'lose_after_lose': 0},
            player2: {'last_result': None, 'win_after_win': 0, 'lose_after_win': 0, 'win_after_lose': 0, 'lose_after_lose': 0}
        }

        for _, match in matches.iterrows():
            winner = match['winner_name']
            loser = match['loser_name']

            # Count für den Gewinner aktualisieren
            if counters[winner]['last_result'] == 'win':
                counters[winner]['win_after_win'] += 1
            elif counters[winner]['last_result'] == 'lose':
                counters[winner]['win_after_lose'] += 1
            counters[winner]['last_result'] = 'win'

            # Count für den Verlierer aktualisieren
            if counters[loser]['last_result'] == 'win':
                counters[loser]['lose_after_win'] += 1
            elif counters[loser]['last_result'] == 'lose':
                counters[loser]['lose_after_lose'] += 1
            counters[loser]['last_result'] = 'lose'

        
        neutral_results.append({
            'Player1': player1,
            'Player2': player2,
            'Player1_win_after_win': counters[player1]['win_after_win'],
            'Player2_win_after_win': counters[player2]['win_after_win'],
            'Player1_lose_after_win': counters[player1]['lose_after_win'],
            'Player2_lose_after_win': counters[player2]['lose_after_win'],
            'Player1_win_after_lose': counters[player1]['win_after_lose'],
            'Player2_win_after_lose': counters[player2]['win_after_lose'],
            'Player1_lose_after_lose': counters[player1]['lose_after_lose'],
            'Player2_lose_after_lose': counters[player2]['lose_after_lose']
        })

    return pd.DataFrame(neutral_results)




### Testen der Funktionen von Spielerpaaren mit mindestens 6 Spielen gegeneinander

In [12]:
frequent_pairings_6_inf = count_player_pairings(mpdf,6,float('inf'))
frequent_pairings_6_inf

,games_count,Player1,Player2
73,6,Aaron Krickstein,Alexander Volkov
75,8,Aaron Krickstein,Amos Mansdorf
77,8,Aaron Krickstein,Andre Agassi
82,12,Aaron Krickstein,Andres Gomez
93,8,Aaron Krickstein,Boris Becker
...,...,...,...
106599,13,Vitas Gerulaitis,Wojtek Fibak
106639,8,Wayne Ferreira,Yevgeny Kafelnikov
106669,6,Wojtek Fibak,Yannick Noah
106670,8,Wojtek Fibak,Zeljko Franulovic


In [13]:
analysis_6_inf = analyze_outcomes_neutral(mpdf, frequent_pairings_6_inf)
analysis_6_inf

,Player1,Player2,Player1_win_after_win,Player2_win_after_win,Player1_lose_after_win,Player2_lose_after_win,Player1_win_after_lose,Player2_win_after_lose,Player1_lose_after_lose,Player2_lose_after_lose
0,Aaron Krickstein,Alexander Volkov,5,0,0,0,0,0,0,5
1,Aaron Krickstein,Amos Mansdorf,2,0,3,2,2,3,0,2
2,Aaron Krickstein,Andre Agassi,0,2,3,2,2,3,2,0
3,Aaron Krickstein,Andres Gomez,1,7,1,2,2,1,7,1
4,Aaron Krickstein,Boris Becker,0,5,1,1,1,1,5,0
...,...,...,...,...,...,...,...,...,...,...
3868,Vitas Gerulaitis,Wojtek Fibak,5,1,3,3,3,3,1,5
3869,Wayne Ferreira,Yevgeny Kafelnikov,2,1,2,2,2,2,1,2
3870,Wojtek Fibak,Yannick Noah,1,2,1,1,1,1,2,1
3871,Wojtek Fibak,Zeljko Franulovic,3,0,2,2,2,2,0,3


### Funktionen zur Visualsisierungen der Spielergruppen

In [14]:
#Funktion zur Visualisierung der Ergebnisse
def visualize_player_scenarios(data, data_name, figsize=(12, 6)):
    

    scenario_counts = pd.DataFrame({
        'Win After Win': data['Player1_win_after_win'] + data['Player2_win_after_win'],
        'Lose After Win': data['Player1_lose_after_win'] + data['Player2_lose_after_win'],
        'Win After Lose': data['Player1_win_after_lose'] + data['Player2_win_after_lose'],
        'Lose After Lose': data['Player1_lose_after_lose'] + data['Player2_lose_after_lose']})
    
    fig = px.box(scenario_counts, points="all")
    fig.update_layout(
    title=f'Verteilung aller Spielszenarien in Spielerpaarungen für {data_name} Begegnungen ({len(data)} Paarungen)',
    yaxis_title='Anzahl der Szenarien pro Paarung',
    xaxis_title='Scenarios'
    )
    fig.show()
        
    

    


In [15]:


def plot_random_pairings(df, data_name,num_pairings=9):
    # Auswahl zufälliger Spielerpaarungen
    random_pairings = df.sample(num_pairings)

    
    fig = make_subplots(rows=3, cols=3, subplot_titles=[f"{row['Player1']} vs {row['Player2']}" for _, row in random_pairings.iterrows()])

    
    scenarios = ['win_after_win', 'lose_after_win', 'win_after_lose', 'lose_after_lose']
    for i, row in enumerate(random_pairings.itertuples(), 1):
        values_player1 = [getattr(row, f'Player1_{scenario}') for scenario in scenarios]
        values_player2 = [getattr(row, f'Player2_{scenario}') for scenario in scenarios]

        
        fig.add_trace(
            go.Bar(
                x=scenarios, 
                y=values_player1, 
                name=f"{row.Player1}'s Outcomes",
                marker_color='blue'
            ),
            row=(i-1)//3 + 1, col=(i-1)%3 + 1
        )

        
        fig.add_trace(
            go.Bar(
                x=scenarios, 
                y=values_player2, 
                name=f"{row.Player2}'s Outcomes",
                marker_color='red'
            ),
            row=(i-1)//3 + 1, col=(i-1)%3 + 1
        )

    
    fig.update_layout(
        height=900, 
        width=900, 
        title_text=f'Vergleich der Spielszenarien für {num_pairings} zufällige Paarungen für {data_name} Begegnungen',
        showlegend=False
    )

    return fig



 Oben haben wir bisher alle Paare die mindestens 6 Spiele gegeneinander gespielt haben, analysiert.
 Nun teilen wir die Paare in Gruppen auf [6-13, 14-20, 20+ (6-Infinity)], um zu sehen ob es Unterschiede gibt.
 Alles wird im vergleich zur 6-Inf Paarungen gestellt.

### 6-13 Spiele

In [16]:
frequent_pairings_6_13 = count_player_pairings(mpdf, 6, 13)
frequent_pairings_6_13

,games_count,Player1,Player2
73,6,Aaron Krickstein,Alexander Volkov
75,8,Aaron Krickstein,Amos Mansdorf
77,8,Aaron Krickstein,Andre Agassi
82,12,Aaron Krickstein,Andres Gomez
93,8,Aaron Krickstein,Boris Becker
...,...,...,...
106599,13,Vitas Gerulaitis,Wojtek Fibak
106639,8,Wayne Ferreira,Yevgeny Kafelnikov
106669,6,Wojtek Fibak,Yannick Noah
106670,8,Wojtek Fibak,Zeljko Franulovic


In [17]:
analysis_6_13 = analyze_outcomes_neutral(mpdf, frequent_pairings_6_13)
analysis_6_13.head()

,Player1,Player2,Player1_win_after_win,Player2_win_after_win,Player1_lose_after_win,Player2_lose_after_win,Player1_win_after_lose,Player2_win_after_lose,Player1_lose_after_lose,Player2_lose_after_lose
0,Aaron Krickstein,Alexander Volkov,5,0,0,0,0,0,0,5
1,Aaron Krickstein,Amos Mansdorf,2,0,3,2,2,3,0,2
2,Aaron Krickstein,Andre Agassi,0,2,3,2,2,3,2,0
3,Aaron Krickstein,Andres Gomez,1,7,1,2,2,1,7,1
4,Aaron Krickstein,Boris Becker,0,5,1,1,1,1,5,0


#### Visualisierung 6-13 (Im Vergleich zu 6-Infinity Begegnungen pro Paarung)

In [18]:
visualize_player_scenarios(analysis_6_13, '6-13'), visualize_player_scenarios(analysis_6_inf, '6-Inf')

(None, None)

In [19]:
len(analysis_6_13)/len(analysis_6_inf)

0.9240898528272656

Wir sehen, dass 6-13 Macthes pro Paarung 92% der Paarungen ausmacht. Daher sind die beiden Boxplots sehr ähnlich. 
Insgesamt sehen wir, dass die verteilung für einen Sieg nach einem Sieg und dementsprechend für eine Niederlage nach einer Niederlage etwas höher ist 
als wenn sich Sieg und Niederlage abwechseln. 
Einen Zusammenhang über das nächste Spiel anhand des vorherigen Spiel ist hier also nicht aussagekräftig zu erkennen.

In [20]:
plot_random_pairings(analysis_6_13, '6-13') 

In [21]:
plot_random_pairings(analysis_6_inf, '6-Inf')

Auch aus 9 Stichproben der 6-13er Begegnugen zu den 6-Inf Begegnungen scheinen die Werte ähnlich zu sein

#### 14-20

In [22]:
frequent_pairings_14_20 = count_player_pairings(mpdf, 14, 20)
frequent_pairings_14_20.head()

,games_count,Player1,Player2
1224,16,Adriano Panatta,Bjorn Borg
1360,16,Adriano Panatta,Manuel Orantes
2113,15,Albert Costa,Alex Corretja
7225,14,Alexandr Dolgopolov,David Ferrer
9644,14,Andre Agassi,Boris Becker


In [23]:
analysis_14_20 = analyze_outcomes_neutral(mpdf, frequent_pairings_14_20)
analysis_14_20.head()

,Player1,Player2,Player1_win_after_win,Player2_win_after_win,Player1_lose_after_win,Player2_lose_after_win,Player1_win_after_lose,Player2_win_after_lose,Player1_lose_after_lose,Player2_lose_after_lose
0,Adriano Panatta,Bjorn Borg,3,7,3,2,2,3,7,3
1,Adriano Panatta,Manuel Orantes,4,4,3,4,4,3,4,4
2,Albert Costa,Alex Corretja,3,4,3,4,4,3,4,3
3,Alexandr Dolgopolov,David Ferrer,0,5,4,4,4,4,5,0
4,Andre Agassi,Boris Becker,8,2,1,2,2,1,2,8


#### Visualisierung 14-20

In [24]:
visualize_player_scenarios(analysis_14_20, '14-20'), visualize_player_scenarios(analysis_6_inf, '6-Inf')

(None, None)

In [25]:
len(analysis_14_20)/len(analysis_6_inf)

0.057836302607797575

Wir sehen, dass es die Paarungen mit 14-20 Spielen gegeneinander inzwischen nur noch 5.7% aller Paare ausmacht. Es kommt also nicht oft dazu dass Paare mehr als 13 Begegnungen gegeneinander haben. 
Allerdings sind die Verteilung der Plots ähnlich zu allen Begegnungen. Nur sind hier die Werte alle nach oben verschoben was an der höheren Anzahl der Spiele liegt.

#### 20+

In [26]:
frequent_pairings_20_inf = count_player_pairings(mpdf, 20, float('inf'))
frequent_pairings_20_inf

,games_count,Player1,Player2
9860,22,Andre Agassi,Michael Chang
9899,34,Andre Agassi,Pete Sampras
12531,21,Andres Gimeno,Ken Rosewall
12573,21,Andres Gimeno,Rod Laver
14127,21,Andy Murray,David Ferrer
...,...,...,...
103304,20,Rod Laver,Tony Roche
103375,27,Roger Federer,Stan Wawrinka
103401,26,Roger Federer,Tomas Berdych
103472,25,Roger Taylor,Tom Okker


In [27]:
analysis_20_inf = analyze_outcomes_neutral(mpdf, frequent_pairings_20_inf)
analysis_20_inf.head()

,Player1,Player2,Player1_win_after_win,Player2_win_after_win,Player1_lose_after_win,Player2_lose_after_win,Player1_win_after_lose,Player2_win_after_lose,Player1_lose_after_lose,Player2_lose_after_lose
0,Andre Agassi,Michael Chang,8,1,6,6,6,6,1,8
1,Andre Agassi,Pete Sampras,5,11,9,8,8,9,11,5
2,Andres Gimeno,Ken Rosewall,0,13,3,4,4,3,13,0
3,Andres Gimeno,Rod Laver,1,9,5,5,5,5,9,1
4,Andy Murray,David Ferrer,11,2,3,4,4,3,2,11


#### Visualisierung 20+

In [28]:
visualize_player_scenarios(analysis_20_inf, '20-Inf'), visualize_player_scenarios(analysis_6_inf, '6-Inf')

(None, None)

In [29]:
len(analysis_20_inf)/len(analysis_6_inf)

0.020914020139426802

Auch der Diagramm-Vergleich zwischen 20-Inf Begegnungen zu 6-Inf Begegnungen zeigt ein ähnliches Muster der Spielszenarien-Verteilung, obwohl 20-Inf Begegnungen nur 1.8% aller Begegnungen ausmachen.


## Fazit

Insgesamt lässt sich also sagen, dass die Verteilung der Spielszenarien über alle Gruppierungen hinweg einen ähnlichen Verlauf nehmen.
In keiner Gruppe gibt es ein Szenario, dass sich erheblich von den anderen Szenarien unterscheidet.
Allgemein ist in jeder Gruppe die Tendenz zu sehen, dass ein Spieler nach einem Sieg erneuet einen Sieg gegen einen bestimmten Gegner einfährt, allerdings nicht sehr viel häufiger wie ein abwechselndes Siegen beider Spieler.
Somit bestätigen die Gruppierungen auch die allgemeine Analyse vom Anfang der Aufgabe
